In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505184 sha256=5c5b4e083e86c27e47ddc2b356eac8be03bb0ac66b3c1222650a92164968b7bc
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [4]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pickle
from flask import Flask, request, jsonify

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# Change path to where your dataset is stored in Drive
file_path = "/content/drive/MyDrive/full_dataset.csv"

import pandas as pd
use_cols = ['title', 'ingredients']  # Load only necessary columns
df = pd.read_csv(file_path, usecols=use_cols)
print(df.head())

Mounted at /content/drive
                   title                                        ingredients
0    No-Bake Nut Cookies  ["1 c. firmly packed brown sugar", "1/2 c. eva...
1  Jewell Ball'S Chicken  ["1 small jar chipped beef, cut up", "4 boned ...
2            Creamy Corn  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...
3          Chicken Funny  ["1 large whole chicken", "2 (10 1/2 oz.) cans...
4   Reeses Cups(Candy)    ["1 c. peanut butter", "3/4 c. graham cracker ...


In [6]:
df['recipe_id'] = df.index

In [7]:
num_users = 5000  # Adjust as needed
df['user_id'] = np.random.randint(1, num_users, size=len(df))
df['rating'] = np.random.randint(1, 6, size=len(df))

In [8]:
print(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   title        object
 1   ingredients  object
 2   recipe_id    int64 
 3   user_id      int64 
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 896.1 MB
None


In [9]:
df['ingredients'] = df['ingredients'].apply(lambda x: eval(x) if isinstance(x, str) else [])

In [10]:
recipes = df[['title', 'ingredients']]

In [11]:
w2v_model = gensim.models.Word2Vec(sentences=recipes['ingredients'], vector_size=100, window=5, min_count=1, workers=4)

In [12]:
def get_recipe_embedding(ingredients):
    vectors = [w2v_model.wv[word] for word in ingredients if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

In [13]:
recipes['embedding'] = recipes['ingredients'].apply(get_recipe_embedding)

<ipython-input-13-075c904f9aad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes['embedding'] = recipes['ingredients'].apply(get_recipe_embedding)


In [14]:
print(df.columns)

Index(['title', 'ingredients', 'recipe_id', 'user_id', 'rating'], dtype='object')


In [15]:
# Verify the new columns
print(df[['user_id', 'recipe_id', 'rating']].head())

   user_id  recipe_id  rating
0     4573          0       1
1     1896          1       5
2     2201          2       5
3     2067          3       1
4      221          4       3


In [16]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

df_sample = df.sample(n=500000, random_state=42)  # Take a 500K row sample

# Collaborative Filtering Model (User-Recipe Matrix Factorization)

In [17]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_sample[['user_id', 'recipe_id', 'rating']], reader)

In [18]:
# Split data for training/testing
trainset, testset = train_test_split(data, test_size=0.2)

In [19]:
# Convert to Surprise's internal dataset format
trainset = data.build_full_trainset()

In [20]:
# Train SVD model
svd = SVD(n_factors=50, n_epochs=10, verbose=True)

In [21]:
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


# Save models

In [22]:
with open("w2v_model.pkl", "wb") as f:
    pickle.dump(w2v_model, f, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
import pickle

with open("w2v_model.pkl", "wb") as f:
    for word in list(w2v_model.wv.index_to_key):  # Iterate over words in vocabulary
        pickle.dump({word: w2v_model.wv[word]}, f, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/w2v_model.pkl"

with open(save_path, "wb") as f:
    pickle.dump(w2v_model, f, protocol=pickle.HIGHEST_PROTOCOL)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
pickle.dump(svd, open("svd_model.pkl", "wb"))

# Flask API

In [ ]:
app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.json
    ingredients = data.get("ingredients", [])
    user_id = data.get("user_id")

    # Content-Based Filtering
    input_embedding = get_recipe_embedding(ingredients)
    recipes['similarity'] = recipes['embedding'].apply(lambda x: cosine_similarity([input_embedding], [x])[0][0])
    content_recs = recipes.nlargest(5, 'similarity')[['title']].to_dict(orient='records')

    # Collaborative Filtering
    collab_recs = df[df['user_id'] == user_id].nlargest(5, 'rating')['recipe_id'].tolist()

    return jsonify({
        "content_based": content_recs,
        "collaborative": collab_recs
    })

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
